

## 函数

函数也是一个对象，叫函数对象。函数名和变量名一样都是引用，函数名后面带个括号才真正实际执行。比如下面不带括号就只是返回了对这个函数对象的引用地址。

    >>> print
    <built-in function print>

要理解函数也是一个对象，比如在下面的例子中，fun刚开始是一个函数列表，然后在for的迭代语句里，意思具体就是multiply这个函数对象，然后接下来又是plus这个函数对象。整个过程是对x\*a然后再加上b。即$a*x +b$

    x = 3
    
    def multiply(x,a):
        return x*a
    
    def plus(x,b):
        return x+b
    
    fun = [multiply , plus]
    para = [3,2]
    for fun,para in zip(fun,para):
        x = fun(x,para)
    print(x)



### 自定义函数

定义函数用def命令，语句基本结构如下：

    def yourfunctionname(para001,para002...):
        do something001
        do something002

### 参数传递问题

函数具体参数的值是通过赋值形式来传递的，这有助于理解后面的不定变量函数。而函数的参数名是没有意义的，这个可以用lambda函式来理解之，def定义的为有名函数，有具体的引用地址，但内部作用原理还是跟lambda无名函式一样，形式参数名是x啊y啊都无所谓。为了说明这点，下面给出一个古怪的例子：

    y=1
    def test(x,y=y):
        return x+y
    print(test(4))

输出结果是5。我们看到似乎函数的形式参数y和外面的y不是一个东西，同时参数的传递是通过赋值形式进行的，那么具体是怎样的呢？具体的解释就是函数的形式参数y是这个函数自己内部的**本地变量**y，和外面的y不一样，更加深入的理解请看下一节（变量作用域问题）。

然后还有：

    >>> x=[1,2,3]
    >>> for x in x:
    ...  print(x)
    ... 
    1
    2
    3

我们知道for语句每进行一次迭代之前也进行了一次赋值操作，所以for语句里面刚开始定义的这个x和外面的x也不是一个东西，刚开始定义的x也是for语句内部的**本地变量**。

想到这里我又想起之前编写removeduplicate函数遇到的一个问题，那就是for语句针对列表这个可变的可迭代对象的工作原理是如何的？具体请看下面的例子：

    >>> lst=[1,2,3,4]
    >>> for x in lst:
    ...  print(x,lst)
    ...  del lst[-1]
    ... 
    1 [1, 2, 3, 4]
    2 [1, 2, 3]

可迭代对象的惰性求值内部机制在我看来很神奇，目前还不太清楚，但从这个例子看来列表的惰性求值并没有记忆内部的数值，只是记忆了（合情合理），然后如果迭代产生了StopIteration异常就终止。

### 变量作用域问题

python的变量作用域和大部分语言比如c语言或lisp语言的概念都类似，就是函数里面是局部变量，一层套一层，里面可以引用外面，外面不可以引用里面。

具体实现机制是每个函数都有自己的命名空间，（和模块类似）就好像一个盒子一样封装着内部的变量。所谓的本地变量和函数有关，或者其他类似的比如for语句；所谓的全局变量和模块有关，更确切的表述是和文件有关，比如说在现在这个文件里，你可以通过导入其他模块的变量名，但实际上模块导入之后那些变量名都引入到这个文件里面来了。

具体实现和类的继承类似也是一种搜索机制，先搜索本地作用域，然后是上一层(def，lambda，for)的本地作用域，然后是全局作用域，然后是内置作用域。更加的直观的说明如下图所示：

![img]({static}/images/python/python-bian-liang-zuo-yong-yu.png)

简单来说python的变量作用域问题就是：盒子套盒子，搜索是从盒子最里面然后往外面寻找，里面可以用外面的变量，外面的不可以用里面的。

#### 内置作用域

内置作用域就是由一个`__builtin__`模块来实现的，python的作用机制最后会自动搜索这个内置模块的变量。这个内置模块里面就是我们前面学习的那些可以直接使用的函数名，比如print，range等等之类的，然后还有一些内置的异常名。

所以我们想到即使对于这些python的内置函数我们也是可以覆盖定义的，事实确实如此：

    >>> abs(-3)
    3
    >>> def abs(x):
    ...  print(x)
    ... 
    >>> abs(3)
    3
    >>> abs(-3)
    -3

以后学习单元测试会接触到mock的概念，其作用机制大体也与之类似就是覆盖掉之前定义的某个对象。

#### global命令

如果希望函数里面定义的变量就是全局变量，在变量声明的时候前面加上**global**命令即可。

通常不建议这么做，除非你确定需要这么做，然后你需要写两行代码才能实现，意思也是不推荐你这么做。

    def test():
        global var
        var= 'hello'
    test()
    print(var)
    
    hello

而且就算你这样做了，这个变量也只能在本py文件中被引用，其他文件用不了。推荐的做法是另外写一个专门用于配置参数的config.py文件，然后那些全局变量都放在里面，如果某个文件要用，就import进来。而对与这个config.py文件的修改会影响所有的py文件配置，这样让全局变量可见可管可控更加通用，才是正确的编程方式。

#### nonlocal命令

nonlocal命令python3之后才出现，这里实现的概念有点类似于lisp语言的闭包(closure技术)，就是如果你有某个需要，需要函数记忆一点自己的状态，同时又不想这个状态信息是全局变量，也不希望用类的方式来实现，那么就可以用nonlocal命令来简单地完成这个任务。

global意味着命名只存在于一个嵌套的模块中，而nonlocal的查找只限于嵌套的def中。要理解nonlocal首先需要理解函数里面嵌套函数的情况------也就是所谓的工厂函数，一个函数返回一个函数对象。比如说

    def add(x):
        x=x
        def action(y):
            return x+y
        return action
    
    >>> add1=add(1)
    >>> add1(5)
    6
    >>> add2=add(2)
    >>> add2(5)
    7

这里的return action是返回一个函数对象，这样add1的实际接口是`def action`那里。熟悉lisp语言的明白，action外面的那个函数的变量叫做自由变量，不过嵌套函数在这里可以引用自由变量。如果我们声明`nonlocal x`，那么就可以修改嵌套函数外面声明的变量了。

    def add(x):
        x=x
        def action(y):
            nonlocal x
            x=x+1
            return x+y
        return action
    
    >>> add2=add(2)
    >>> add2(5)
    8
    >>> add2(5)
    9
    >>> add2(5)
    10

然后我们看到这个生产出来的函数具有了运行上的状态性，实际上通过类也能构建出类似的效果，不过对于某些问题可能闭包方式处理显得更适合一些。

下面给出一个稍微合理点的例子：

    def myrange(n):
        i=n
        def action():
            nonlocal i
            while i>0:
                i=i-1
                return i
        return action
    
    >>> myrange5=myrange(5)
    >>> myrange5()
    4
    >>> myrange5()
    3
    >>> myrange5()
    2
    >>> myrange5()
    1
    >>> myrange5()
    0
    >>> myrange5()
    >>> 

下面给出类似的类的实现方法：

    class myrange:
        def __init__(self,n):
            self.i=n
        def action(self):
            while self.i > 0:
                self.i -= 1
                return self.i
    
    >>> myrange5=myrange(5)
    >>> 
    >>> myrange5.action()
    4
    >>> myrange5.action()
    3
    >>> myrange5.action()
    2
    >>> myrange5.action()
    1
    >>> myrange5.action()
    0
    >>> myrange5.action()
    >>> 

我们看到从编码思路上基本上没什么差异，可以说稍作修改就可以换成类的实现版本。推荐一般使用类的实现方法。但有的时候可能用类来实现有点不伦不类和大材小用了。这里就不做进一步讨论了，闭包思想是函数编程中很重要的一个思想，学习了解一下也好。

### 参数和默认参数

定义的函数圆括号那里就是接受的参数，如果参数后面跟个等号，来个赋值语句，那个这个赋的值就是这个参数的默认值。比如下面随便写个演示程序：

    def test(x='hello'):
        print(x)
    test()
    test('world')
    
    hello
    world

### 不定参量函数

我们在前面谈到sum函数只接受一个列表，而不支持这样的形式：sum(1,2,3,4,5)。现在我们设计这样一个可以接受不定任意数目参量的函数。首先让我们看看一种奇怪的赋值方式。

#### 序列解包赋值

**NOTICE:**
python2不支持本小节讨论的序列解包赋值。不过python2的函数定义中是支持
`*args` 这种写法的。

    >>> a,b,*c=1,2,3,4,5,6,7,8,9
    >>> print(a,b,c,sep=' | ')
    1 | 2 | [3, 4, 5, 6, 7, 8, 9]
    >>> a,*b,c=1,2,3,4,5,6,7,8,9
    >>> print(a,b,c,sep=' | ')
    1 | [2, 3, 4, 5, 6, 7, 8] | 9
    >>> *a,b,c=1,2,3,4,5,6,7,8,9
    >>> print(a,b,c,sep=' | ')
    [1, 2, 3, 4, 5, 6, 7] | 8 | 9

带上一个星号\*的变量变得有点类似通配符的味道了，针对后面的序列（数组，列表，字符串），它都会将遇到的元素收集在一个列表里面，然后说是它的。

for语句也支持序列解包赋值，也是将通配到的的元素收集到了一个列表里面，如：

    for (a,*b,c) in [(1,2,3,4,5,6),(1,2,3,4,5),(1,2,3,4)]:
        print(b)
    
    [2, 3, 4, 5]
    [2, 3, 4]
    [2, 3]

#### 函数中的通配符

    >>> def test(*args):
    ...  print(args)
    ... 
    >>> test(1,2,3,'a')
    (1, 2, 3, 'a')

我们看到类似上面序列解包赋值中的带星号表通配的概念，在定义函数的时候写上一个带星号的参量（我们可以想象在函数传递参数的时候有一个类似的序列解包赋值过程），在函数定义里面，这个args就是接受到的参量组成的*元组*。

#### mysum函数

    def mysum(*args):
        return sum(args[:])
    
    print(mysum(1,2,3,4,5,6))
    
    21

这样我们定义的可以接受任意参数的mysum函数，如上所示。具体过程就是将接受到的args（已成一个元组了），然后用sum函数处理了一下即可。

#### 任意数目的可选参数

在函数定义的写上带上两个星号的变量\*\*args，那么args在函数里面的意思就是接受到的可选参数组成的一个字典值。

    >>> def test(**args):
    ...  return args
    ... 
    >>> test(a=1,b=2)
    {'b': 2, 'a': 1}

我们看到利用这个可以构建出一个简单的词典对象生成器。

#### 解包可迭代对象传递参数

之前\*args是在函数定义中，然后通配一些参数放入元组中。这里是在函数调用中，针对可迭代对象，可以用一个\*星号将其所包含的元素迭代出来，然后和参数一一对应赋值。

    >>> map = map(lambda x:x+2,[1,2,3])
    >>> print(*map)
    3 4 5
    >>> print(*[1,2,3])
    1 2 3

##### 最简单的打印文件命令

前面说到文件也是一个可迭代对象，然后如果在这里解包文件对象将是一个最简单的打印文件命令，简单得惊天地泣鬼神了\...

    print(*open('test.py'))

#### 解包字典成为关键字参数

和上面的类似，通过\*\*args语法可以将某个字典对象解包成为某个函数的关键字参数。还是以上面那个函数f为例子：

    >>> def f(a,b,c=3):
    ...  print(a,b,c)
    >>> f(**{'c':6,'b':4,'a':2})
    2 4 6
    >>> f(1,2,5)
    1 2 5

这个例子也告诉我们不是可选参数的a和b同样也可以通过这种字典形式复制。

### 参数的顺序

老实说一般参数，可选参数（关键字参数），任意（通配）参数，任意（通配）关键字参数所有这些概念混在一起非常的让人困惑。就一般的顺序是：

1.  一般参数，这个如果有 ，然后通过位置一一对应分配参数。

2.  关键字参数，匹配一些关键字参数。

3.  通配一般参数，其他额外的非关键字的参数分配到\*args元组里面。

4.  通配关键字参数，其他额外的关键字参数分配到\*\*kwargs字典里面， 。

具体如下所示：

    def test(x, y, c=1, d=1, *args, **kwargs):
        print(x, y, c, d, args, kwargs)

这种写法也是python2和python3兼容的。


#### keyword-only参数
python3新加入了一个keyword-only参数（读者记住这不是关键字参数就行了），如下所示：

    def test(x, y, c=1, d=1, *args, z=None ,**kwargs):
        print(x, y, c, d,args, kwargs,z)

看上面的例子，这个keyword-only参数是个极容易和keyword参数或者我们常说的关键字参数混淆的东西，这个keyword-only参数也确实是类似关键字参数，但它不能像常规的关键字参数那样按照位置赋值，而必须明确的指定名字赋值。

这个keyword-only参数的标志就是跟在那个星号后面。如下所示，你也别把那个z认为是个一般参数了，它只是一个还没有赋予默认值的keyword-only参数。

    def test(x, y, c=1, d=1, *args, z ,**kwargs):
        print(x, y, c, d,args, kwargs,z)

然后有的人就只想用keyword-only参数，对具体通配一般参数根本不感兴趣，会这样写：

    def test(x, y, c=1, d=1, *, z ,**kwargs):
        print(x, y, c, d,args, kwargs,z)

这样写的话就没有通配一般参数了。

keyword-only的参数的用处就是其是一个不能通过不指定名字而赋值的关键字参数，或者说如果你需要某个关键字参数在后面的使用中必须明确给出名字来使用，那么就可以使用keyword-only参数。

只是有一点，python2不支持这个东西，python2要实现类似的效果如下所示：

    def test(x, y, **kwargs):
        a = kwargs.pop('a')
        b = kwargs.pop('b', False) # 第二个参数得到默认参数的效果
        
        if kwargs:
            raise TypeError(Unexpected kwargs: {0}'.format(kwargs))

异常信息随便写的，在这里不是重点。

### 生成器函数

一般函数的定义使用return语句，如果使用yield语句，我们可以构建出一个生成器函数，

```text
>>> def test(x):
...    for i in range(x):
...        yield 2*i+ 1
... 
>>> test(3)
<generator object test at 0xb704348c>
>>> [x for x in test(3)]
[1, 3, 5]
>>> [x for x in test(5)]
[1, 3, 5, 7, 9]
```

生成器函数返回的是生成器对象（generator object），通过yield这样的形式定义出来的生成器函数返回了一个生成器对象和range对象类似，都是描述性可迭代对象，里面的元素并不立即展开，而是请求一次运算一次，所以这种编程风格对内存压力很小，主要适合那些迭代元素特别多的时候的情况吧。

上面的test函数我们就可以简单理解为 `2x+1` ，其中` 0<=x<n`（赋的值）。

下面给出一个问题作为练习：描述素数的生成器函数。
这是网上流行的素数检验函数，效率还是比较高的了。

    def isprime(n):
        if n ==2:
            return True
        #按位与1，前面一定都是0个位数如果是1则
        #是奇数则返回1则真则假，如果是偶数则返回
        #0则假则真
        elif n<2 or not n & 1:
            return False
        #埃拉托斯特尼筛法...
    #查一个正整数N是否为素数，最简单的方法就是试除法，
    #将该数N用小于等于N**0.5的所有素数去试除，
    #若均无法整除，则N为素数
        for x in range(3,int(n**0.5)+1,2):
            if n % x == 0:
                return False
        return True

然后我们给出两种形式的素数生成器函数，其中prime2的意思是范围到（to）那里。而prime(n)的意思是到第几个素数。我们知道生成器函数是一种惰性求值运算，然后yield每迭代一次函数运算一次（即产生一次yield），但这种机制还是让我觉得好神奇。

    def prime2(n):
        for x in range(n):
            if isprime(x):
                yield x
    
    def prime(n):
        i=0
        x=1
        while i<n:
            if isprime(x):
                i +=1
                yield x
            x +=1

在加载这些函数之后我们可以做一些检验：

    >>> isprime(479)
    True
    >>> [x for x in prime2(100)]
    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, ........]
    >>> [x for x in prime2(1000) if 100< x < 200]
    [101, 103, 107, 109, 113, 127, 131, 137, 139, 149, .......]
    >>> len([x for x in prime2(10000) if -1 < x < 3572])
    500
    >>> [x for x in prime(1)]
    [2]
    >>> [x for x in prime(2)]
    [2, 3]

### 递归函式 

虽然递归函式能够在某种程度上取代前面的一些循环或者迭代程序结构，不过不推荐这么做。这里谈及递归函式是把某些问题归结为数学函数问题，而这些问题常常用递归算法更加直观（不一定高效）。比如下面的菲波那奇函数：

    def fib(n):
        if n==0:
            return 1
        if n==1:
            return 1
        else:
            return fib(n-1)+fib(n-2)
            
    for x in range(5):
        print(fib(x))
    
    1
    1
    2
    3
    5

我们可以看到，对于这样专门的数学问题来说，用这样的递归算法来表述是非常简洁易懂的。至于其内部细节，我们可以将上面定义的fib称之为函式，函式是一种操作的模式，然后具体操作就是复制出这个函式（函数或者操作都是数据），然后按照这个函式来扩展生成具体的函数或者操作。

下面看通过递归函式来写阶乘函数，非常的简洁，我以为这就是最好最美的方法了。

    def fact(n):
        if n == 0:
            return 1
        else:
            return n*fact(n-1)
            
    print(fact(0),fact(10))
    
    1 3628800

#### 什么时候用递归？

最推荐使用递归的情况是这样的情况，那就是一份工作（或函数）执行一遍之后你能够感觉到虽然所有的工作没有做完，但是已经做了一小部分了，有了一定的进展了，就好比是蚂蚁吞大象一样，那么这个时候你就可以使用递归思想了。其次有的时候有那么一种情况虽然表面上看似乎并没有什么进展，但事情在发展，你能感受到有一个条件最终将会终止程序从而得到一个输出，那么这个时候就可以用递归。

递归思想最核心的两个概念就是一做了一小部分工作，你能感觉到做着做着事情就会做完了；二有一个终止判断最终将会起作用。

其实通过递归函式也可以实现类似for的迭代结构，不过我觉得递归函式还是不应该滥用。比如下面通过递归函式生成一种执行某个操作n次的结构：

    def dosomething(n):
        if n==0:
            pass
        elif n==1:
            print('do!')
        else:
            print('do!')
            return dosomething(n-1)
    
    print(dosomething(5))
    
    do!
    do!
    do!
    do!
    do!
    None

可以看到，如果把上面的print语句换成其他的某个操作，比如机器人向前走一步，那么这里dosomething换个名字向前走(5)就成了向前走5步了。

#### lisp的car-cdr递归技术

在lisp语言中，
car-cdr递归技术是很重要的一门技术，它的特长就是遍历随意嵌套的列表结构可以同一对列表中的每一个元素执行某种操作。

首先我们来看下面的例子，一个把任意嵌套列表所有元素放入一个列表中的函数：

    lst = [[1,2,[3]],[4,[5,[[[[10],11]]]],(1,2,3)],[{'a','b','c'},8,9]]
    
    def is_list(thing):
        return isinstance(thing, list)
    
    def flatten(iter):
        templst = []
        for x in iter:
            if not is_list(x):
                templst.append(x)
            else:
                templst += flatten(x)
        return templst
    
    print(flatten(lst))
    
    [1, 2, 3, 4, 5, 10, 11, (1, 2, 3), {'c', 'b', 'a'}, 8, 9]

这个函数的逻辑是如果是最小元素对象不是列表，那么收集进列表；如果不是，那么把它展开，这里就是调用的原函数继续展开函式。

上面的例子严格意义上来讲还不算lisp的经典car-cdr递归技术，下面给出一个典型的例子，就是复制任意嵌套结构的列表。当然列表的copy方法就可以做这个工作，这里主要通过这个例子来进一步深入car-cdr技术。

    def is_list(thing):
        return isinstance(thing, list)
    
    def copy_list(lst):
        if  not  is_list(lst):
            return lst
        elif lst == []:
            return []
        else:
            return [copy_list(lst[0])] + copy_list(lst[1:])
    
    print(copy_list([1,[2,6],3]))

这种嵌套列表的复制以及后面的修改等等操作，最合适的就是lisp的car-cdr技术了，但我不得不承认，这种递归写法是递归函式里面最难懂的了。

不管怎么严格，在这个基础之上，因为第一个if
not的语句中传递下来的lst实际上已经是非列表的其他元素了，然后我们可以进行一些其他修改操作，这样在保持原列表的复杂嵌套的基础上，等于遍历的对列表中的所有元素进行了某种操作。

比如所有元素都平方：

    def square(x):
        return x**2
    
    def square_list(lst):
        if  not  is_list(lst):
            return square(lst)
        elif lst == []:
            return []
        else:
            return [square_list(lst[0])] + square_list(lst[1:])
    
    print(square_list([1,[2,6],3]))

我们可以想像更加复杂功能的函数作用于列表中所有的元素同时又不失去原列表复杂的嵌套结构，lisp的car-cdr这种技术了解一下吧，但是不是一定要使用复杂的嵌套结构呢？也许没有必要吧。。

### lambda函式

python中的函数作为一个对象一般是通过 `def` 语句来定义的，定义之后该函数对象和函数名变量已经绑定在一起了。但实际上python中的函数作为一个对象名字不是那么重要的：

```
def add(x,y):
...     return x+y
... 
add
<function add at 0x000001B952B770D0>

add2 = add
del add
add2(1,2)
3
add
Traceback (most recent call last):
  File "<input>", line 1, in <module>
NameError: name 'add' is not defined
add2.__name__
'add'
```

而lambda λ表达式可以简单理解为没有名字的函数对象：

```
add = lambda x,y: x+y
add
<function <lambda> at 0x000001B952B77378>
add.__name__
'<lambda>'
add(1,2)
3
type(add)
<class 'function'>
type(add2)
<class 'function'>
```

如上所示我们看到，不管是通过def定义的函数还是lambda函式在python里面都是函数对象，或者说都属于function class。具体调用和使用也很类似，除了lambda函数定义的函数对象那个 `__name__` 只是默认的 `<lambda>` 。

lambda函数在某些地方会用到，一般是在将函数作为参数传递的情况下，某些简短的函数动作没必要另外再想个函数名字的应用场景。

### print函数

print函数接受任意的参量，逐个打印出来。然后它还有一些关键字参数：

- **sep**：默认值是 `' '` ，也就是一个空格，如果修改为空字符串，那么逐个打印出来的字符之间就没有间隔了。
- **end**：默认值是 `'\n'` ，为print函数执行完打印的字符，默认是打印一个换行符。
- **file**默认值是 `sys.stdout` ，也就是在终端显示，你可以修改为某个文件变量，这样直接往某个文件里面输出内容。
